In [8]:
import pandas as pd
import numpy as np

from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier


In [9]:
df_src = pd.read_csv('data/titanic/train.csv')
df_src.head(1)
#df_src.isna().sum()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,NaN,S


In [10]:
def prepare_dataframe(df, train = True):
    if train:
        y = df['Survived']
        X = df.drop(['Survived'], axis = 1)
    else: 
        y = None
        X = df 
    X = X.fillna({'Age': X.Age.median()})
    X = X.fillna(0)
    X.set_index('PassengerId', inplace=True)
    X = X[['Pclass', 'Sex', 'Age', 'Fare']]
    X = pd.get_dummies(X)
    return X, y

In [11]:
X, y = prepare_dataframe(df_src, train = True)

In [19]:
# более сильную GridSearch
clf = DecisionTreeClassifier()
params = {'criterion' : ['gini','entropy'],
          'max_depth': [4,5,6,10],
          'min_samples_split': [5,10,20,35],
          'min_samples_leaf' : [5,10,20,35],
          'min_impurity_decrease': [0.03, 0.05,0.1,0.2,0.3]}

search = GridSearchCV(clf, params, cv = 5)
search.fit(X,y)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=DecisionTreeClassifier(class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort=False, random_state=None,
                                              splitter='best'),
             iid='warn', n_jobs=None,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [4, 5, 6, 10],
                        

In [24]:
search.best_params_, search.best_score_

({'criterion': 'gini',
  'max_depth': 4,
  'min_impurity_decrease': 0.05,
  'min_samples_leaf': 5,
  'min_samples_split': 5},
 0.7867564534231201)

In [22]:
df_test = pd.read_csv('data/titanic/test.csv')
X_res, _ = prepare_dataframe(df_test, train = False)

X_res['Survived'] = search.best_estimator_.predict(X_res)
X_res[['Survived']].to_csv('data/titanic/result.csv')